# Предсказание победы в игре Dota 2

## Необходимые библиотеки и классы

In [1]:
import sklearn
import pandas as pd
import numpy as np
from sklearn import model_selection, ensemble
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

## Функция для подсчёта метрики rocauc

In [2]:
def rocauc (estimator, X, y):
    pred = estimator.predict_proba(X)[:, 1]
    score = sklearn.metrics.roc_auc_score(y, pred)
    return score

## Анализируемый датафрейм

#### Считываю таблицу с признаками из файла features.csv

In [3]:
data = pd.read_csv("data/features.csv", index_col='match_id')
print(data)
for i in range(data.shape[1]):
    if data.count()[i] != data.shape[0]:
        print(data.columns[i])

          start_time  lobby_type  r1_hero  r1_level  r1_xp  r1_gold  r1_lh  \
match_id                                                                     
0         1430198770           7       11         5   2098     1489     20   
1         1430220345           0       42         4   1188     1033      9   
2         1430227081           7       33         4   1319     1270     22   
3         1430263531           1       29         4   1779     1056     14   
4         1430282290           7       13         4   1431     1090      8   
...              ...         ...      ...       ...    ...      ...    ...   
114402    1450265551           1       47         4   1706     1198     17   
114403    1450277704           0       43         4   1793     1416     17   
114404    1450291848           1       98         4   1399      540      1   
114405    1450292986           1      100         3   1135      766      6   
114406    1450313370           7       50         3   1053      

C:\Users\aniri\AppData\Local\Temp\ipykernel_12800\1331978564.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data.count()[i] != data.shape[0]:
C:\Users\aniri\AppData\Local\Temp\ipykernel_12800\1331978564.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data.count()[i] != data.shape[0]:
C:\Users\aniri\AppData\Local\Temp\ipykernel_12800\1331978564.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data.count()[i] != d

first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time


C:\Users\aniri\AppData\Local\Temp\ipykernel_12800\1331978564.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data.count()[i] != data.shape[0]:
C:\Users\aniri\AppData\Local\Temp\ipykernel_12800\1331978564.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data.count()[i] != data.shape[0]:
C:\Users\aniri\AppData\Local\Temp\ipykernel_12800\1331978564.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data.count()[i] != d

radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


C:\Users\aniri\AppData\Local\Temp\ipykernel_12800\1331978564.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data.count()[i] != data.shape[0]:
C:\Users\aniri\AppData\Local\Temp\ipykernel_12800\1331978564.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data.count()[i] != data.shape[0]:
C:\Users\aniri\AppData\Local\Temp\ipykernel_12800\1331978564.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data.count()[i] != d

## Градиентный бустинг

### Подготовка данных для применения градиентного бустинга

#### Удаляю признаки, связанные с итогами матча

In [4]:
data = data.fillna(0)
data = data.sample(frac=1)
y_train = data.iloc[0:int(data.shape[0]/10), 103]
X_train = data.iloc[0:int(data.shape[0]/10), 0:102]

#### Целевую переменную содержит столбец: 'radiant_win'

### Применение градиентного бустинга с кросс-валидацией по 5-ти блокам

In [5]:
#градиентный бустинг от 10 до 100

kf = sklearn.model_selection.KFold(n_splits=5, shuffle=True)
est = np.arange(10, 110, 10)
for i in range(len(est)):
    clf = sklearn.ensemble.GradientBoostingClassifier(learning_rate=0.9, n_estimators=est[i], verbose=0, max_depth=2)
    metric_c = sklearn.model_selection.cross_val_score(clf, X_train, y_train, scoring=rocauc, cv=kf)
    print(sum(metric_c)/len(metric_c))

0.6652415831592562
0.6788854431614046
0.678107354456231
0.6738558393260439
0.6785805154064064
0.6746820764989441
0.6699331414356047
0.6713738150655237
0.6664341879849923
0.6692470514975328


## Логистическая регрессия

### Подготовка данных для применения логистической регрессии

In [6]:
#логистическая регрессия

y_train = data['radiant_win']
X_train = data.iloc[:, 0:102]

X_train.index = [i for i in range(data.shape[0])]

print(X_train.nunique()[2])

108


C:\Users\aniri\AppData\Local\Temp\ipykernel_12800\4291082206.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(X_train.nunique()[2])


### Применение логистической регрессии

### Подсчёт кол-ва героев

In [7]:
# N — количество различных героев в выборке
X_pick = np.zeros((data.shape[0], X_train.nunique()[2]))

C:\Users\aniri\AppData\Local\Temp\ipykernel_12800\1503049096.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  X_pick = np.zeros((data.shape[0], X_train.nunique()[2]))


### Перенумерация id героев

In [8]:
#Перенумерация id героев
num1 = [i+1 for i in range(23)]
num2 = [i for i in range(25, 107)]
num3 = [109, 110]
num4 = [112]

for j in range(2, 75, 8):
    for i in range(data.shape[0]):
        if data.iloc[i, j] in num1:
            data.iloc[i, j] = data.iloc[i, j]
        elif data.iloc[i, j] in num2:
            data.iloc[i, j] = data.iloc[i, j] - 1
        elif data.iloc[i, j] in num3:
            data.iloc[i, j] = data.iloc[i, j] - 3
        else: data.iloc[i, j] = data.iloc[i, j] - 4

### Реализация мешка слов

In [9]:
#реализация мешка слов
for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

X_train_drop = X_train.drop(columns=['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'])

X_pick = pd.DataFrame(X_pick)
X_train_new = X_train_drop.join(X_pick)
X_train_new.columns = X_train_new.columns.astype(str)

### Нормализация признаков для применения логистической регрессии

In [10]:
#нормализация признаков
scaler = sklearn.preprocessing.StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_new)

### Выбор лучшего параметра C l2-регуляризации

In [11]:
#выбор лучшего параметра C l2-регуляризации
grid = {'C': np.power(2.0, np.arange(-5, 5))}
logreg = sklearn.linear_model.LogisticRegression()
gs = sklearn.model_selection.GridSearchCV(logreg, grid, scoring=rocauc, cv=kf)
gs.fit(X_train_scaled, y_train)
logreg = sklearn.linear_model.LogisticRegression(C=gs.best_params_['C'])
metric_l = sklearn.model_selection.cross_val_score(logreg, X_train_scaled, y_train, scoring=rocauc, cv=kf)
print(sum(metric_l)/len(metric_l))

0.7519127285110406


### Применение к тестовым данным

In [12]:
#теперь всё это прогоняем на тестовых данных
data_test = pd.read_csv("data/features_test.csv", index_col='match_id')
data_test = data_test.fillna(0)
X_test = data_test.iloc[:, 0:102]
X_test.index = [i for i in range(data_test.shape[0])]

X_pick = np.zeros((data_test.shape[0], X_test.nunique()[2]))

num1 = [i+1 for i in range(23)]
num2 = [i for i in range(25, 107)]
num3 = [109, 110]
num4 = [112]

for j in range(2, 75, 8):
    for i in range(data_test.shape[0]):
        if data_test.iloc[i, j] in num1:
            data_test.iloc[i, j] = data_test.iloc[i, j]
        elif data_test.iloc[i, j] in num2:
            data_test.iloc[i, j] = data_test.iloc[i, j] - 1
        elif data_test.iloc[i, j] in num3:
            data_test.iloc[i, j] = data_test.iloc[i, j] - 3
        else: data_test.iloc[i, j] = data_test.iloc[i, j] - 4

for i, match_id in enumerate(data_test.index):
    for p in range(5):
        X_pick[i, data_test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data_test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

X_test_drop = X_test.drop(columns=['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'])
X_pick = pd.DataFrame(X_pick)
X_test_new = X_test_drop.join(X_pick)
X_test_new.columns = X_test_new.columns.astype(str)

X_test_scaled = scaler.transform(X_test_new)

C:\Users\aniri\AppData\Local\Temp\ipykernel_12800\2472328998.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  X_pick = np.zeros((data_test.shape[0], X_test.nunique()[2]))


### Обучение алгоритма логистической регрессии

In [13]:
#обучение алгоритма логистической регрессии
logreg.fit(X_train_scaled, y_train)
#logreg.predict_proba(X_test_scaled)

array([[0.17568649, 0.82431351],
       [0.24620994, 0.75379006],
       [0.81217344, 0.18782656],
       ...,
       [0.76388267, 0.23611733],
       [0.37104664, 0.62895336],
       [0.57453646, 0.42546354]])

### Вывод предсказаний о победе Radiant

In [14]:
#вывод предсказаний о победе radiant
pred = {'match_id': [data_test.index[i] for i in range(X_test_scaled.shape[0])], 'radiant_win': logreg.predict_proba(X_test_scaled)[:, 1]}
pred = pd.DataFrame(data=pred)
pred.set_index('match_id')

,radiant_win
match_id,
6,0.824314
7,0.753790
10,0.187827
13,0.859375
16,0.241295
...,...
114369,0.718872
114377,0.631344
114378,0.236117
